In [4]:
#importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')

import re


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
#loading the dataset
df=pd.read_csv('Final_Data_Cleaning.csv')

In [37]:
df.head()

,Unnamed: 0,Gender,Country,Hour,Day,Weekday,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,Lang,text,clean_text,translated_text,tokens
0,0,Unknown,Albania,7,12,Friday,True,339.0,127.0,0.0,44.0,0.0,en,"""RT @AdrianRusso82: Our Innovation Lab is offi...",our innovation lab officially open click learn...,innovation officially open click learn tech ja...,"['innovation', 'officially', 'open', 'click', ..."
1,1,Male,Albania,11,7,Thursday,False,87.0,0.0,0.0,22.0,0.0,en,Now Open AWS Asia Pacific (Seoul) Region via ...,now open aws asia pacific seoul region via are...,open asia pacific seoul region aresysadmin,"['open', 'asia', 'pacific', 'seoul', 'region',..."
2,2,Male,Albania,6,12,Tuesday,False,87.0,0.0,0.0,22.0,0.0,en,A Beginner's Guide to Scaling to 11 Million+ U...,a beginners guide scaling million users amazon...,beginners guide scaling million users amazons ...,"['beginner', 'guide', 'scaling', 'million', 'u..."
3,3,Male,Albania,10,12,Tuesday,False,87.0,0.0,0.0,22.0,0.0,en,Bridging AWS and Azure environments via VPN vi...,bridging aws azure environment via vpn via are...,bridging azure environment aresysadmin,"['bridging', 'azure', 'environment', 'aresysad..."
4,4,Male,Albania,9,21,Thursday,False,85.0,0.0,0.0,21.0,0.0,en,ELK on AWS ElasticSearch Service + ElasticBean...,elk aws elasticsearch service elasticbeanstalk...,elasticsearch service elasticbeanstalk laravel...,"['elasticsearch', 'service', 'elasticbeanstalk..."


In [38]:
# dropping columns
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [39]:
df['Lang'].value_counts()

Lang
en    93239
es     1800
fr     1094
tr      899
ja      570
de      569
_u      437
in      421
tl      417
pt      231
ko      205
nl      174
it      148
zh       91
th       79
pl       75
cs       74
fi       51
et       42
ru       41
ro       35
cy       29
da       28
ht       26
sv       26
hu       16
no       13
lt        8
ar        6
lv        6
hi        3
sl        2
uk        2
km        1
eu        1
Name: count, dtype: int64

In [40]:
df.shape

(100859, 16)

#### At this point we are decided to drop the rows by using `Lang` column which are not translated in english as most of them are translated in it.

In [41]:
# List of substrings to remove
substrings_to_remove = ['_u','es', 'ht','in','it','ja','nl','pt','ro','tl','tr','zh']
# Create a regex pattern to match any of the substrings
pattern = '|'.join(substrings_to_remove)
# Remove rows where 'Lang' contains any of the substrings
df = df[~df['Lang'].str.contains(pattern)]

In [42]:
df.shape

(95610, 16)

# Data Engineering

In [43]:
# dropping columns
df.drop(['Gender','Country','Hour','Day','Weekday','IsReshare','Reach','RetweetCount','Likes','Klout','Lang','text'],axis=1,inplace=True)

In `Sentiment` column, the values are not well defined. So, We are dividing them into 3 categories and then applying mapping.

In [46]:
# the values are not well defined. So, We are dividing them into 3 categories and then applying mapping.
df['Mood'] = df['Sentiment'].apply(lambda x: 
                                       ('Positive' if x > 0 
                                             else ('Neutral' if x == 0 
                                                   else  'Negative')))

In [61]:
# Mapping dictionary
mood_mapping = {
    'Neutral': 0,
    'Positive': 1,
    'Negative': 2
}

df['Mood'] = df['Mood'].map(mood_mapping)

In [62]:
# Mapping dictionary
mood_mapping = {
    0: 'Neutral',
    1: 'Positive',
    2: 'Negative'
}

df['Sentiment'] = df['Mood'].map(mood_mapping)

In [64]:
# Handle missing values by replacing NaNs with an empty string
df['translated_text'] = df['translated_text'].fillna('')

# Convert non-string values to string
df['translated_text'] = df['translated_text'].astype(str)

In [65]:
df.shape

(95610, 5)

In [68]:
df.head(5)

,Sentiment,clean_text,translated_text,tokens,Mood
0,Neutral,our innovation lab officially open click learn...,innovation officially open click learn tech ja...,"['innovation', 'officially', 'open', 'click', ...",0
1,Neutral,now open aws asia pacific seoul region via are...,open asia pacific seoul region aresysadmin,"['open', 'asia', 'pacific', 'seoul', 'region',...",0
2,Neutral,a beginners guide scaling million users amazon...,beginners guide scaling million users amazons ...,"['beginner', 'guide', 'scaling', 'million', 'u...",0
3,Neutral,bridging aws azure environment via vpn via are...,bridging azure environment aresysadmin,"['bridging', 'azure', 'environment', 'aresysad...",0
4,Neutral,elk aws elasticsearch service elasticbeanstalk...,elasticsearch service elasticbeanstalk laravel...,"['elasticsearch', 'service', 'elasticbeanstalk...",0


In [67]:
df.to_csv("Final_Data_Engineering_Data.csv",index=True)